<a href="https://colab.research.google.com/github/Eliane-orlandin/Trilha_para_elas_IA/blob/main/C%C3%B3pia_de_Aula_guiada_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O que vamos aprender?

* Carregar um modelo de linguagem em memória
* Configurando o Hardware (CPU vs GPU)
* Controlando a variabilidade da resposta com o parâmetro Temperature
* Interagindo com o Dataset



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

## Instalação das bibliotecas

In [ ]:
!pip install -U pip datasets

## Carregando o modelo em memória
**Pipeline**
https://huggingface.co/docs/transformers/en/main_classes/pipelines



>

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="unsloth/Llama-3.2-1B-Instruct")


In [ ]:
messages = [
    {"role": "user", "content": "Olá, tudo bem?"},
]
pipe(messages)

## Controlando a variabilidade da resposta com o parâmetro: Temperature

>[Configurações para a geração do texto](https://huggingface.co/docs/transformers/v4.52.3/en/main_classes/text_generation#transformers.GenerationConfig)

<img src=https://res.cloudinary.com/dwppkb069/image/upload/v1683736290/tips/images-03-temperature.mp4/03-temperature_30_03-25000-if-temperature-is-like-almost-0--were-going-to-have-a-very-sharp-peaked-distribution_wprxqo.png width="500">

In [ ]:
messages = [
    {"role": "user", "content": "Me conte uma boa piada"},
]
pipe(messages)[0]["generated_text"][1]['content']

In [ ]:
messages = [
    {"role": "user", "content": "Me conte uma boa piada"},
]
pipe(messages, temperature=0.01)[0]["generated_text"][1]['content']

## Carregar dataset :: b2w-reviews01

In [ ]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("ruanchaves/b2w-reviews01")

ds = pd.DataFrame(ds['train'])
ds.head()


##1 - Indicando produtos a um amigo

> Dado o título do produto e a review, devemos verificar se o modelo consegue prever corretamente se o autor do comentário indicaria ou não o produto

---







In [ ]:
ds[['review_title', 'review_text']].head()

In [ ]:
example = ds.sample(n=1, random_state=21)
print('Titulo: ', example['review_title'].to_list()[0])
print()
print('Review: ', example['review_text'].to_list()[0])

In [ ]:
messages = [
      {"role": "user", "content": """
        Você é uma assistente que vai receber uma revisão de um produto e deve dizer se o autor da review recomenda ou não o produto para um amigo
        Titulo: BOM
        Review: SÓ DEMOROU A CHEGAR, MAS JÁ COMPREI OUTRAS COISA E NÃO TIVE PROBLEMAS, FOI A PRIMEIRA VEZ QUE ATRASA, DE RESTO TUDO BEM.
        """},
]
output = pipe(messages,temperature=0.01)[0]["generated_text"][1]['content']
print('Saida do modelo: ', output)

In [ ]:
messages = [
      {"role": "user", "content": """
        Você é uma assistente que vai receber uma revisão de um produto e deve dizer se o autor da review recomenda ou não o produto para um amigo.
        Gere uma explicação passo a passo.
        Titulo: BOM
        Review: SÓ DEMOROU A CHEGAR, MAS JÁ COMPREI OUTRAS COISA E NÃO TIVE PROBLEMAS, FOI A PRIMEIRA VEZ QUE ATRASA, DE RESTO TUDO BEM.
        """},
]
output = pipe(messages,temperature=0.01)[0]["generated_text"][1]['content']
print('Saida do modelo: ', output)

## 2: Gerando descrição automática do produto
> Dado o nome do produto e sua categoria, devemos instruir o modelo a gerar uma descrição automática do produto.

---



In [ ]:
messages = [
    {"role": "user", "content": """Dado o nome do produto abaixo, gere uma descrição sucinta para atrair a venda desse produto.
                                  Titulo: Fone De Ouvido Eb-201bk Preto Lite; Categoria: Áudio"""},
]
output = pipe(messages,temperature=0.01)[0]["generated_text"][1]['content']
output

In [ ]:
messages = [
    {"role": "user", "content": """Dado o nome do produto abaixo, gere uma descrição sucinta para atrair a venda desse produto para o seguinte perfil de compradores:
                                  Jovens de 15 a 20 anos que gostam de jogar video-game;
                                  Titulo: Fone De Ouvido Eb-201bk Preto Lite; Categoria: Áudio"""},
]
output = pipe(messages,temperature=0.01)[0]["generated_text"][1]['content']
output

In [ ]:
messages = [
    {"role": "user", "content": """Dado o nome do produto abaixo, gere uma descrição sucinta para atrair a venda desse produto para o seguinte perfil de compradores:
                                  Adultos de mais de 40 anos a 60 anos;
                                  Titulo: Fone De Ouvido Eb-201bk Preto Lite; Categoria: Áudio"""},
]
output = pipe(messages,temperature=0.01)[0]["generated_text"][1]['content']
output

## 3: Gerando o resumo de revisões de um produto
Dado as top-3 avaliações aleátorias de um determinado produto, devemos instruir ao modelo a gerar um resumo das avaliações.

Essa prática é semelhante ao empregado em RAG (Retrieval-Augmented Generation), isto é, o modelo utiliza de fontes externas (infos de um DB, busca da internet, documentos internos) para compor a saída final.

---



In [ ]:
from collections import Counter
Counter(ds['product_name'].to_list()).most_common(3)


In [ ]:
prod_name = 'Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Platinum'
filter_prod = ds[ds['product_name']==prod_name].sample(3 , random_state=15)
filter_prod

In [ ]:
messages = [
      {"role": "user", "content": """
        Dada as reviews de um produto abaixo, gere um único resumo geral.
        Review 1:
        Adorei o produto
        Como faço para fazer seguro do celular ? quero fazer e como proceder

        Review 2:
        Excelente custo e benefício
        Ótimo aparelho. Surpreende na qualidade da mantagem, acabamento e material aplicado na carcaça. Mas dá umas travada de vez em quando. Não é o aparelho mais rápido do mundo mas tem excelente desempenho. Até o momento não tivemos nem um problema

        Review 3:
        Moto g5S
        Compra e entrega ocorreram conforme esperado. Estou plenamente satisfeito com a compra."""},
]
output = pipe(messages,temperature=0.01)[0]["generated_text"][1]['content']
print('Saida do modelo: ', output)